In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [2]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

In [3]:
train_data = pd.read_csv('../data/coggle_train.csv')

In [4]:
train_data.loc[0]

id                                                 103245
text    Michel Mayor used astrometric data provided by...
cat                                             Astronomy
Name: 0, dtype: object

In [5]:
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))
s = train_data.loc[0]['text'].translate(str.maketrans('','',string.punctuation))
tokens = word_tokenize(s)

In [6]:
filtered_sentence = [w for w in tokens if not w in stop_words] 

In [7]:
stemmed_words = []
for word in filtered_sentence:
    stemmed_words.append(ps.stem(word))

## Sklearn

In [100]:
test_data = pd.read_csv('../data/coggle_test.csv')

In [101]:
train_corpus = [x['text'] for idx,x in train_data.iterrows()]
test_corpus = [x['text'] for idx,x in test_data.iterrows()]

In [102]:
corpus = train_corpus + test_corpus

In [103]:
from sklearn.feature_extraction.text import CountVectorizer

In [104]:
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(corpus)

In [105]:
X.shape

(41639, 142944)

In [106]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X)

/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [107]:
X_tfidf.shape

(41639, 142944)

In [108]:
y_train_cat = [x['cat'] for idx,x in train_data.iterrows()]

In [109]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train_enc = le.fit_transform(y_categories)

In [110]:
train_data.shape

(33391, 3)

In [111]:
X_train_tfidf = X_tfidf[:33391]

In [112]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train_tfidf, y_train_enc, random_state=7)

In [113]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)

In [114]:
y_pred = clf.predict(X_train_tfidf)

In [115]:
print(np.mean(y_pred == y_train))

0.17477763469198287


## Testing

In [116]:
X_test_tfidf = X_tfidf[33391:]

In [117]:
predicted = clf.predict(X_test_tfidf)